In [1]:
from __future__ import print_function
import collections
import os
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Flatten, Dropout, TimeDistributed, Reshape, Lambda
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam, SGD
from keras import backend as K
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import numpy as np
import pdb

/usr/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [43]:
train_pathN = "dataset/no_punctuation/1M/wiki_00"
train_pathP = "dataset/cured_text/1M/wiki_00"
valid_pathN = "dataset/no_punctuation/1M/wiki_01"
valid_pathP = "dataset/cured_text/1M/wiki_01"
test_pathN = "dataset/no_punctuation/1M/wiki_02"
test_pathP = "dataset/cured_text/1M/wiki_02"
models_path = "models"
symbols = [".", "?", "¿", "!", "¡", ",", ":", ";"]

In [44]:
def read_words(filename):
    with tf.gfile.GFile(filename, "r") as f:
        return f.read().replace("\n", " <eos>").split()

In [46]:
print(len(read_words(train_pathN)))
print(len(read_words(train_pathP)))
#read_words(train_pathN)

171127
188953


In [47]:
def build_vocab(filename):
    data = read_words(filename)

    counter = collections.Counter(data)        
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))        

    words, _ = list(zip(*count_pairs))     
    word_to_id = dict(zip(words, range(len(words))))
    
    return word_to_id

In [48]:
def file_to_word_ids(filename, word_to_id):
    data = read_words(filename)
    return [word_to_id[word] for word in data if word in word_to_id]

In [49]:
def load_data():
    # build the complete vocabulary, then convert text data to list of integers
    word_to_id = build_vocab(train_pathP)
    train_dataN = file_to_word_ids(train_pathN, word_to_id)
    train_dataP = file_to_word_ids(train_pathP, word_to_id)
    valid_dataN = file_to_word_ids(valid_pathN, word_to_id)
    valid_dataP = file_to_word_ids(valid_pathP, word_to_id)
    test_dataN = file_to_word_ids(test_pathN, word_to_id)
    test_dataP = file_to_word_ids(test_pathP, word_to_id)
    vocabulary = len(word_to_id)
    reversed_dictionary = dict(zip(word_to_id.values(), word_to_id.keys()))

    #print(train_data[:5])
    #print(word_to_id)
    #print(vocabulary)
    #print(" ".join([reversed_dictionary[x] for x in train_data[:10]]))
    return train_dataN, train_dataP, valid_dataN, valid_dataP, test_dataN, test_dataP, vocabulary, reversed_dictionary

train_dataN, train_dataP, valid_dataN, valid_dataP, test_dataN, test_dataP, vocabulary, reversed_dictionary = load_data()

In [50]:
print(len(train_dataP))
print(" ".join([reversed_dictionary[x] for x in train_dataN[100:110]]))
print(" ".join([reversed_dictionary[x] for x in train_dataP[100:110]]))

188953
Sobirá — y por el norte con Francia — con
con España — con las comarcas catalanas de Cerdaña ,


In [51]:
class KerasBatchGenerator(object):

    def __init__(self, dataN, dataP, num_steps, batch_size, vocabulary, skip_step=5):
        self.dataN = dataN
        self.dataP = dataP
        self.num_steps = num_steps
        self.batch_size = batch_size
        self.vocabulary = vocabulary
        # this will track the progress of the batches sequentially through the
        # data set - once the data reaches the end of the data set it will reset back to zero
        self.cnt1 = 0
        self.cnt2 = 0
        # skip_step is the number of words which will be skipped before the next batch is skimmed from the data set
        self.skip_step = skip_step

    def generate(self):
        x = np.zeros((self.batch_size, self.num_steps))
        y = np.zeros((self.batch_size, self.num_steps, self.vocabulary))
        while True:
            for i in range(self.batch_size):
                if self.cnt1 + self.num_steps >= len(self.dataN):
                    # reset the index back to the start of the data set
                    self.cnt1 = 0
                    self.cnt2 = 0
                x[i, :] = self.dataN[self.cnt1 : self.cnt1 + self.num_steps]
                temp_y = self.dataP[self.cnt2 : self.cnt2 + self.num_steps]
                print()
                print(" ".join([reversed_dictionary[x] for x in self.dataN[self.cnt1 : self.cnt1 + self.num_steps]]))
                print(" ".join([reversed_dictionary[x] for x in self.dataP[self.cnt2 : self.cnt2 + self.num_steps]]))
                
                # convert all of temp_y into a one hot representation
                y[i, :, :] = to_categorical(temp_y, num_classes=self.vocabulary)
                
                self.cnt1 += self.skip_step
                for i in range(self.skip_step):
                    while reversed_dictionary[self.dataP[self.cnt2]] in symbols:                        
                        self.cnt2 += 1
                    self.cnt2 += 1
                
            yield x, y

In [52]:
num_steps = 30
batch_size = 20

train_data_generator = KerasBatchGenerator(train_dataN, train_dataP, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps)
valid_data_generator = KerasBatchGenerator(valid_dataN, valid_dataP, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps)
hidden_size = 500
use_dropout=True

In [53]:
model = Sequential()
model.add(Embedding(vocabulary, hidden_size, input_length=num_steps))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(LSTM(hidden_size, return_sequences=True))
if use_dropout:
    model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(vocabulary)))
model.add(Activation('softmax'))

optimizer = Adam()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 30, 500)           10771500  
_________________________________________________________________
lstm_13 (LSTM)               (None, 30, 500)           2002000   
_________________________________________________________________
lstm_14 (LSTM)               (None, 30, 500)           2002000   
_________________________________________________________________
dropout_7 (Dropout)          (None, 30, 500)           0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, 30, 21543)         10793043  
_________________________________________________________________
activation_7 (Activation)    (None, 30, 21543)         0         
Total params: 25,568,543
Trainable params: 25,568,543
Non-trainable params: 0
________________________________________________________________

In [54]:
#1 to train, 2 to test
run_opt = 1
#checkpointer = ModelCheckpoint(filepath = models_path + "/model-{epoch:02d}.hdf5", verbose=1)
num_epochs = 50

In [ ]:
if run_opt == 1:
    model.fit_generator(train_data_generator.generate(), len(train_dataP)//(batch_size*num_steps), num_epochs,
                        validation_data=valid_data_generator.generate(),
                        validation_steps=len(valid_dataP)//(batch_size*num_steps))
    # model.fit_generator(train_data_generator.generate(), 2000, num_epochs,
    #                     validation_data=valid_data_generator.generate(),
    #                     validation_steps=10)
    model.save(models_path + "/final_model.hdf5")

Epoch 1/50

Andorra <eos> Andorra oficialmente Principado de Andorra ( ) es un pequeño país soberano del suroeste de Europa Constituido en Estado independiente de derecho democrático y social cuya forma de
Andorra <eos> Andorra , oficialmente Principado de Andorra ( ) , es un pequeño país soberano del suroeste de Europa . Constituido en Estado independiente , de derecho , democrático

gobierno es el coprincipado parlamentario Su territorio está organizado en siete parroquias con una población total de <num> <num> habitantes ( <num> ) Su capital es Andorra la Vieja <eos>
gobierno es el coprincipado parlamentario . Su territorio está organizado en siete parroquias , con una población total de <num> <num> habitantes ( <num> ) . Su capital es Andorra

Tiene <num> km² de extensión territorial y está situado en los Pirineos entre España y Francia con una altitud media de <num> msnm Limita por el sur con España —
Tiene <num> km² de extensión territorial y está situado en los Pirineos , entre 

el Reino de Francia <eos> Como consecuencia de los conflictos entre Roger Bernardo III de Foix y Pedro de Urtx el <num> de septiembre de <num> se firma el primer
, el Reino de Francia . <eos> Como consecuencia de los conflictos entre Roger Bernardo III de Foix y Pedro de Urtx , el <num> de septiembre de <num> se

" pariaje " en Lérida que fija los límites del poder de cada señor representa la fundación del Principado de Andorra y define las obligaciones de los andorranos en materia
" pariaje " en Lérida , que fija los límites del poder de cada señor , representa la fundación del Principado de Andorra y define las obligaciones de los andorranos

de diezmos y asuntos militares por lo que se formó un " condominio feudal " aunque esto no le preocupa a la Corona de Aragón ya que el condado de
de diezmos y asuntos militares , por lo que se formó un " condominio feudal " , aunque esto no le preocupa a la Corona de Aragón , ya que

Foix era vasallo suyo y Andorra estaba dentro de las fronteras


Civil Española y la consiguiente victoria del bando franquista además del estallido de la II Guerra Mundial y la invasión de Francia por parte de los alemanes de Hitler dejó
Civil Española y la consiguiente victoria del bando franquista , además del estallido de la II Guerra Mundial y la invasión de Francia por parte de los alemanes de Hitler

al país en una difícil tesitura amparada por el Gobierno de Vichy dirigido por el Mariscal Petain tuvo que simpatizar forzosamente con los regímenes de Hitler y de Francisco Franco
al país en una difícil tesitura , amparada por el Gobierno de Vichy dirigido por el Mariscal Petain , tuvo que simpatizar forzosamente con los regímenes de Hitler y de

para mantener su neutralidad aprovechando esta situación para organizar cadenas de evasión destinadas a pasar judíos perseguidos por la Gestapo así como oficiales y soldados aliados hacia España <eos> Finalmente
para mantener su neutralidad , aprovechando esta situación para organizar cadenas de evasió


de Ordino y La Massana para finalmente en la ciudad de Escaldes - Engordany confluir ambos ríos y formar el Gran Valira con un recorrido de <num> km y un
de Ordino y La Massana para finalmente , en la ciudad de Escaldes - Engordany , confluir ambos ríos y formar el Gran Valira , con un recorrido de <num>

caudal anual medio de <num> m / s Este último en su descenso hacia el sur acaba desembocando en el río Segre que a su vez es afluente del Ebro
caudal anual medio de <num> m / s . Este último , en su descenso hacia el sur , acaba desembocando en el río Segre que , a su vez

<eos> Andorra tiene más de <num> lagos Los más representativos son el lago de Juclar cuya superficie es la más extensa de todos los lagos del Principado con <num> hectáreas
. <eos> Andorra tiene más de <num> lagos . Los más representativos son el lago de Juclar , cuya superficie es la más extensa de todos los lagos del Principado

y que durante la época estival y por la consecuente falta de agua puede avistarse com


julio de <num> Este acuerdo dependía de la ratificación andorrana la cual sucedió en abril de <num> momento en que entró en vigor En octubre de <num> las autoridades andorranas
julio de <num> Este acuerdo dependía de la ratificación andorrana , la cual sucedió en abril de <num> momento en que entró en vigor . En octubre de <num> las

pospusieron la emisión de monedas en euro hasta <num> año en el que finalmente fue emitida la primera serie <eos> Tradicionalmente Andorra emitía monedas denominadas « Diners » que no
pospusieron la emisión de monedas en euro hasta <num> año en el que finalmente fue emitida la primera serie . <eos> Tradicionalmente Andorra emitía monedas denominadas « Diners » que

tenían curso legal sino que eran con fines de coleccionismo Estas monedas podrán seguir siendo emitidas <eos> La supervisión de los bancos andorranos y demás entidades financieras corresponde al Instituto
tenían curso legal sino que eran con fines de coleccionismo . Estas monedas podrán seguir 

...

  2/314 [..............................] - ETA: 50:08 - loss: 9.9763 - categorical_accuracy: 0.0575      
la segunda más desarrollada e importante en Sudamérica Según el Banco Mundial su PIB nominal es el <num>º del mundo Debido a su importancia geopolítica y económica es uno de
la segunda más desarrollada e importante en Sudamérica . Según el Banco Mundial , su PIB nominal es el <num>º del mundo . Debido a su importancia geopolítica y económica

los tres estados soberanos latinoamericanos que forma parte del denominado Grupo de los <num> e integra además el grupo de los NIC o nuevos países industrializados <eos> Es uno de
los tres estados soberanos latinoamericanos que forma parte del denominado Grupo de los <num> e integra además el grupo de los NIC o nuevos países industrializados . <eos> Es uno

los únicos países del continente americano –junto a Estados Unidos y Canadá– que ha alcanzado óptimos niveles de Investigación y Desarrollo en el campo de las ciencias Su capacidad tecnológ

  4/314 [..............................] - ETA: 41:01 - loss: 9.9736 - categorical_accuracy: 0.0567
de un nivel cultural paleolítico que tres mil años atrás incorporaron los primeros aportes culturales mesolíticos y neolíticos Hasta la época de la conquista y de la colonización europea el
de un nivel cultural paleolítico que tres mil años atrás incorporaron los primeros aportes culturales mesolíticos y neolíticos . Hasta la época de la conquista y de la colonización europea

territorio argentino ha estado ocupado por diversos pueblos originarios con diferentes organizaciones sociales que se pueden dividir en tres grupos principales <eos> En los siglos XIV y XV el Imperio
territorio argentino ha estado ocupado por diversos pueblos originarios , con diferentes organizaciones sociales que se pueden dividir en tres grupos principales : <eos> En los siglos XIV y XV

incaico conquistó parte de las actuales provincias de Jujuy Salta Catamarca el extremo oeste de la provincia de Tucumán parte 

  6/314 [..............................] - ETA: 38:03 - loss: 9.9557 - categorical_accuracy: 0.0619
a ser administrados por el gobierno unitario de Buenos Aires La misma organizó el llamado Congreso de Oriente en Concepción del Uruguay del cual aún se discute si alcanzó a
a ser administrados por el gobierno unitario de Buenos Aires . La misma organizó el llamado Congreso de Oriente en Concepción del Uruguay , del cual aún se discute si

proclamar — como se proponía — la independencia de España <eos> El <num> de julio de <num> en la ciudad de San Miguel de Tucumán reunido el congreso de diputados
proclamar — como se proponía — la independencia de España . <eos> El <num> de julio de <num> en la ciudad de San Miguel de Tucumán , reunido el congreso

de las provincias del noroeste y centro - oeste del país y de la de Buenos Aires junto con algunos diputados exiliados del Alto Perú proclamó la independencia de las
de las provincias del noroeste y centro - oeste del país y de la de Buenos Ai

  8/314 [..............................] - ETA: 36:51 - loss: 9.8526 - categorical_accuracy: 0.0665
es la sede del gobierno Le siguen en población Les Escaldes Encamp San Julián de Loria y La Massana <eos> Andorra está dividida en siete territorios llamados parroquias Son meramente
es la sede del gobierno . Le siguen en población Les Escaldes , Encamp , San Julián de Loria y La Massana . <eos> Andorra está dividida en siete territorios

administrativas sin autonomía propia a su vez tienen divisiones menores Son Canillo Encamp Ordino La Massana Andorra la Vieja San Julián de Loria y Escaldes - Engordany <eos> La actividad
administrativas sin autonomía propia ; a su vez , tienen divisiones menores . Son : Canillo , Encamp , Ordino , La Massana , Andorra la Vieja , San Julián

humana se centra en el valle transversal noreste - suroeste que a partir del Puerto de Envalira ( <num> m ) desciende hasta los <num> m cuando el río Valira
humana se centra en el valle transversal noreste - suroest

 10/314 [..............................] - ETA: 36:25 - loss: 9.6262 - categorical_accuracy: 0.0658
hab ) <eos> El idioma oficial es el catalán lengua habitual del <num> de la población El castellano es la segunda lengua más hablada en el principado con un <num>
hab . ) . <eos> El idioma oficial es el catalán , lengua habitual del <num> de la población . El castellano es la segunda lengua más hablada en el

de la población que la tienen como lengua habitual El portugués es la lengua habitual del <num> de la población y el francés del <num> <eos> Sin embargo el castellano
de la población que la tienen como lengua habitual . El portugués es la lengua habitual del <num> de la población y el francés del <num> . <eos> Sin embargo

es la lengua materna o primera lengua del <num> de la población mientras que el catalán lo es para el <num> <eos> En Andorra conviven tres sistemas educativos el francés
es la lengua materna o primera lengua del <num> de la población , mientras que el catalán lo e

In [ ]:
if run_opt == 2:
    model = load_model(models_path + "/final_model.hdf5")    
    example_training_generator = KerasBatchGenerator(train_dataN, train_dataP, num_steps, 1, vocabulary,
                                                     skip_step=1)
    print("Training data:")    
    num_predict = 10
    true_print_out = "Actual words: "
    pred_print_out = "Predicted words: "
    for i in range(num_predict):
        data = next(example_training_generator.generate())
        prediction = model.predict(data[0])
        predict_word = np.argmax(prediction[:, num_steps-1, :])
        true_print_out += reversed_dictionary[train_dataP[num_steps + i]] + " "
        pred_print_out += reversed_dictionary[predict_word] + " "
    print(true_print_out)
    print(pred_print_out)
    # test data set    
    example_test_generator = KerasBatchGenerator(test_dataN, test_dataP, num_steps, 1, vocabulary,
                                                     skip_step=1)
    print("Test data:")    
    num_predict = 10
    true_print_out = "Actual words: "
    pred_print_out = "Predicted words: "
    for i in range(num_predict):
        data = next(example_test_generator.generate())
        prediction = model.predict(data[0])
        predict_word = np.argmax(prediction[:, num_steps - 1, :])
        true_print_out += reversed_dictionary[test_dataP[num_steps + i]] + " "
        pred_print_out += reversed_dictionary[predict_word] + " "
    print(true_print_out)
    print(pred_print_out)